<a href="https://colab.research.google.com/github/agroimpacts/VegMapper/blob/dev-calval-simplify/process_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a single training/validation/test set from multiple Collect Earth projects



### Table of Contents

* [Overview](#overview)
* [Set-up](#setup)
* [Sample preparation](#sample-prep)
    * [Read-in, reshape, and recode](#reshape-recode)
    * [Simplify the classes](#simplify)
    * [Calculate sample agreement](#agreement)
* [Split the dataset](#split)
    * [Combine and convert to spatial](#combine)

## Overview <a class="anchor" id="overview"></a>
This notebook demonstrates how several Collect Earth Online projects can be:

1. Provide functionality to check the structure and validity of user input for modeling; Yet, users are responsible for providing data in good format.
2. Re-code the class values and rename the column names.
2. Merged into a single dataset that provide a single label for each sample point and an estimate of label uncertainty;
3. Split into training, validation, and test (or map reference) samples.

The data used in this demonstration are the results of three Collect Earth Online projects that were captured over the Department of Ucayali, Peru. Each project represents the efforts of an individual (or group of individuals working in the same project) to label 1350 points, classifying each into 1 of 4 classes: not oil palm; young oil palm; mature oil palm;  unsure. The datasets preserve all the information from these projects, although user email addresses were anonymized.

## Sample preparation <a class="anchor" id="sample-prep"></a>
Load packages, setup configuations, define a helper function...

In [ ]:
# from label_utils import load_csv, subset_cols, rename_cols,\
#     check_exclusive, recode, combine_labelers, get_mode_and_occurence

#@title (RUN) Setup code
## Mount Drive
from google.colab import drive
root = '/content/gdrive'
drive.mount(root)

## Clone and/or update VegMapper
import os
# from datetime import datetime as dt
import pandas as pd
from sklearn.model_selection import train_test_split

repo_path = f"{root}/MyDrive/repos"
clone_path = 'https://github.com/agroimpacts/VegMapper.git'
if not os.path.exists(repo_path):
    print(f"Making {repo_path}")
    os.makedirs(repo_path, exist_ok=True)

if not os.path.exists(f"{repo_path}/VegMapper"):
    !git -C "{repo_path}" clone "{clone_path}"
else:
    !git -C "{repo_path}/VegMapper" pull

os.chdir(f"{repo_path}/VegMapper")

# Import sample_utils function
from vegmapper.calval.label_utils import *

In [ ]:
# import N survey file(s) into a list (N>=1)
fs = ["calval/data/ceo-survey-user1.csv",
      "calval/data/ceo-survey-user2.csv",
      "calval/data/ceo-survey-user3.csv"]

# Set rename dict for renaming column names
# {"old":"new",...}
rename_dict = {"plot_id":"Point_ID",
               "pl_cluster":"Clust",
               "center_lat":"Lat",
               "center_lon":"Lon",
               "Oil Palm?:Young Oil Palm":"Young",
               "Oil Palm?:Mature Oil Palm":"Mature",
               "Oil Palm?:Not Oil Palm":"Not",
               "Oil Palm?:Not Sure":"NotSure"}

# Set re-code dict for land cover classes
recode_dict = {"Young":1,"Mature":1,"Not":0,"NotSure":3}

# Set columns to keep:
# key_col and label_name are used for joining users's datasets,
# columns in useful_col will not participate in joining
# and come from the first user instead to avoid repetition.
key_col = ["Point_ID", "Clust"]
label_name = "labeler"
useful_col = ["Lat", "Lon"]

#Set random seed for train/validation/reference split
seed = 999

In [ ]:
# A helper function to process a csv file
def process_csv(csv_path):
    """
    A csv processing pipeline. This function takes a single csv file
    and let it pass through a sequence of our pre-defined functions
    return: a pandas dataframe of the processed csv.
    """
    print("processing: {}".format(csv_path))
    df = load_csv(csv_path)
    df = rename_cols(df, rename_dict)
    check_exclusive(df, csv_path)

    # if you want to combine Young and Mature, just recode both to be 1.
    df = recode(df, recode_dict, label_name)

    df = subset_cols(df, [*key_col,  *useful_col, label_name])

    return df

### Read-in, reshape, and recode classes <a class="anchor" id="reshape-recode"></a>
The first step was to combine the three datasets into a single dataset, with the columns from each of the three CEO projects, and to recode the four classes into a single column with values 0 (not oil palm),  1 (young oil palm), 2 (mature oil palm), 3 (unsure). At this step, we end up with 3 columns, 1 per completed CEO project: `cl1` = samples from project 1, `cl2` = samples from project 2, `cl3` = samples from project 3. Each column contains the recoded classes (note the renaming of the columns is done in the next code chunk).

In [ ]:
# process ceo-survey-users one by one
dats = list(map(process_csv, fs))

# combine three datasets into one
combined = combine_labelers(dats,by=["Point_ID","Clust"], label_name = label_name)

In [ ]:
combined

### Simplify the classes <a class="anchor" id="simplify"></a>

In this step, a single classification is created by finding the modal class for each sample point across the 3 groups' results. This creates a new `class` column, which provides the class from the majority opinion.

We repeat this same step again after first collapsing, within each of the `cl1:cl3` columns, the two oil palm classes into a single *oil palm* class with value = 1--*not oil palm* remains 0, and *unsure* remains 3. The modal function was re-run to create a new consensus class, called `class2`. We recommend that `class2` be used for modelling, while `class` may be useful for understanding error patterns.

### Calculate sample agreement <a class="anchor" id="agreement"></a>
The next step was to calculate some agreement metrics across the three groups' samples. The primary approach was to calculate the proportion of labelling teams that selected the modal class. Since there were just three teams in this example, values were either 0.333, 0.667, 1. This agreement was calculated across for both the original classification scheme (class: 0-3) and the simplifed scheme (), with columns `agree` and `agree2` providing the respective proportions for each observation.

In [ ]:
combined[['mode', 'mode_agreement']] = combined[
    ["labeler_1","labeler_2","labeler_3"]
].apply(get_mode_and_occurence, axis=1, result_type='expand')
pd.set_option('display.max_rows', None)
#print(combined)

# we can set the mode to -9999 if there is no agreement (mode_freq = 1/num_labelers)
combined.loc[combined['mode_agreement'] <= 1/3, 'mode'] = -9999

combined = combined.drop(combined[combined['mode'] == -9999].index)
print(combined.shape)

combined.head()

We can then calculate the average agreement per sample to get a sense of the uncertainty in labels for each class, for all 4 classes

In [ ]:
agreement = combined.groupby("mode").mean()
agreement = agreement.rename(columns={"mode_agreement": "mean agreement"})
print(agreement[['mean agreement']])

In [ ]:
combined.head()

And for the reduced set of classes

## Split the dataset <a class="anchor" id="split"></a>

Here we split the dataset into three parts for model training (60% of the sample), validation (20%), and final assessment (the 20% set aside as the test or map reference dataset).

The splits are confined to the usable sample, which is defined as samples not falling into class 3 and those with at least 2/3 observers agreeing on the class. This decision is made based on the simplified sample scheme (class2), rather than the full scheme (class). The resulting splits are denoted in a column called `usage` (this is distinct from the column `use`, which was used to filter out unusable observations).

Values of "unusable" in the `usage` column indicate observations that were not usable because of their low agreement or uncertain class.  They are included here for completeness, and in case they help with evaluation


In [ ]:
train, test = train_test_split(combined, test_size=0.4, train_size=0.6,
                               random_state=seed)

val, ref = train_test_split(test, test_size=0.5, train_size=0.5,
                            random_state=seed)

out = pd.concat(
    [train.assign(usage="train"), val.assign(usage="validate"),
     ref.assign(usage="map_reference/test")]
).reset_index(drop=True)
out.head()

### Combine and export to csv <a class="anchor" id="combine"></a>

The ineligible portion of the sample is also added back for completeness

In [ ]:
#@title (RUN) Export sample
gdrive_folder = input(f"Enter the name of the output folder: \n\n")
csv_name = input(f"Enter the name of the output csv file: \n\n")

outpath = f"{root}/MyDrive/{gdrive_folder}/{csv_name}"
with open(outpath, 'w') as f:
    out.to_csv(f, float_format='{:f}'.format, encoding='utf-8',
                    index=False)

print('file exported')

And their locations on a map

In [ ]:
plot_sample = out.copy()

usage_dict = {'train': 1, 'validate': 2, "map_reference/test": 3, "unusable": 4}
plot_sample = plot_sample.replace({'usage': usage_dict})

print(plot_sample['usage'].unique())

rcParams['figure.figsize'] = 10, 10
plot_sample.plot.scatter(x='Lon', y='Lat', c='usage', s=12, cmap='viridis')
None